In [1]:
import numpy as np
import requests  
from bs4 import BeautifulSoup  
import bs4  
import re  
import traceback  
from setuptools.package_index import HREF 
from lxml import etree
class StockCode(object):
    def __init__(self):
        self.start_url = "http://quote.eastmoney.com/stocklist.html#sh"
        self.headers = {
            "User-Agent": ":Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36"
        }

    def parse_url(self):
        response = requests.get(self.start_url, headers=self.headers)
        if response.status_code == 200:
            return etree.HTML(response.content)

    def get_code_list(self, response):
        node_list = response.xpath('//*[@id="quotesearch"]/ul[1]/li')
        code_list = []
        for node in node_list:
            try:
                code = re.match(r'.*?\((\d+)\)', etree.tostring(node).decode()).group(1)
                print(code)
                code_list.append(code)
            except:
                continue
        return code_list

    def run(self):
        html = self.parse_url()
        return self.get_code_list(html)

In [2]:
code=StockCode()

In [3]:
code_list = code.run()

201000
201001
201002
201003
201004
201005
201008
201009
201010
202001
202003
202007
203007
203008
203009
203016
203017
203018
203019
203020
203021
203031
203032
203033
203040
203041
203042
203043
203044
203045
203049
203050
203051
203052
203053
203054
204001
204002
204003
204004
204007
204014
204028
204091
204182
500001
500002
500003
500005
500006
500007
500008
500009
500010
500011
500013
500015
500016
500017
500018
500019
500021
500025
500028
500029
500035
500038
500039
500056
500058
501000
501001
501002
501003
501005
501007
501008
501009
501010
501011
501012
501015
501016
501017
501018
501019
501020
501021
501022
501023
501025
501026
501027
501028
501029
501030
501031
501032
501035
501036
501037
501038
501039
501040
501043
501045
501046
501047
501048
501050
501051
501106
501300
501301
501302
501303
501305
501306
502000
502001
502002
502003
502004
502005
502006
502007
502008
502010
502011
502012
502013
502014
502015
502016
502017
502018
502020
502021
502022
502023
502024
502025
502026

In [4]:
code_list_sz = [code for code in code_list if int(code)>=600000 and int(code)<=699999]
print(code_list_sz)

['600000', '600001', '600002', '600003', '600004', '600005', '600006', '600007', '600008', '600009', '600010', '600011', '600012', '600015', '600016', '600017', '600018', '600019', '600020', '600021', '600022', '600023', '600025', '600026', '600027', '600028', '600029', '600030', '600031', '600033', '600035', '600036', '600037', '600038', '600039', '600048', '600050', '600051', '600052', '600053', '600054', '600055', '600056', '600057', '600058', '600059', '600060', '600061', '600062', '600063', '600064', '600065', '600066', '600067', '600068', '600069', '600070', '600071', '600072', '600073', '600074', '600075', '600076', '600077', '600078', '600079', '600080', '600081', '600082', '600083', '600084', '600085', '600086', '600087', '600088', '600089', '600090', '600091', '600092', '600093', '600094', '600095', '600096', '600097', '600098', '600099', '600100', '600101', '600102', '600103', '600104', '600105', '600106', '600107', '600108', '600109', '600110', '600111', '600112', '600113',

AttributeError: module 'requests' has no attribute 'encoding'

In [28]:
class Download_HistoryStock(object):
    import codecs

    def __init__(self, code):
        self.code = code
        self.start_url = "http://quotes.money.163.com/trade/lsjysj_" + self.code + ".html"
        print(self.start_url)
        self.headers = {
            "User-Agent": ":Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36"}

    def parse_url(self):

        response = requests.get(self.start_url)
        print(response.status_code)
        if response.status_code == 200:
            return etree.HTML(response.content)
        return False

    def get_date(self, response):
        start_date = ''.join(response.xpath('//input[@name="date_start_type"]/@value')[0].split('-'))
        end_date = ''.join(response.xpath('//input[@name="date_end_type"]/@value')[0].split('-'))
        return start_date,end_date

    def download(self, start_date, end_date):
        print('Now downloading stock code:',self.code)
        
        download_url = "http://quotes.money.163.com/service/chddata.html?code=0"+self.code+"&start="+start_date+"&end="+end_date+"&fields=TCLOSE;HIGH;LOW;TOPEN;LCLOSE;CHG;PCHG;TURNOVER;VOTURNOVER;VATURNOVER;TCAP;MCAP"
        data = requests.get(download_url)
        data.encoding = ''
        title = 'D:/data/' + self.code + '.csv'
        f = open(title, 'wb')

        for chunk in data.iter_content(chunk_size=10000):
            if chunk:
                f.write(chunk)
                
        f.close()
        
        print('Download completed')

    def run(self):
        try:
            self.download('20170101', '20171231')
        except Exception as e:
            print(e)

In [31]:
for code in code_list_sz:
    download = Download_HistoryStock(code)
    download.run()

http://quotes.money.163.com/trade/lsjysj_600000.html
Now downloading stock code: 600000
Download completed
http://quotes.money.163.com/trade/lsjysj_600001.html
Now downloading stock code: 600001
Download completed
http://quotes.money.163.com/trade/lsjysj_600002.html
Now downloading stock code: 600002
Download completed
http://quotes.money.163.com/trade/lsjysj_600003.html
Now downloading stock code: 600003
Download completed
http://quotes.money.163.com/trade/lsjysj_600004.html
Now downloading stock code: 600004
Download completed
http://quotes.money.163.com/trade/lsjysj_600005.html
Now downloading stock code: 600005
Download completed
http://quotes.money.163.com/trade/lsjysj_600006.html
Now downloading stock code: 600006
Download completed
http://quotes.money.163.com/trade/lsjysj_600007.html
Now downloading stock code: 600007
Download completed
http://quotes.money.163.com/trade/lsjysj_600008.html
Now downloading stock code: 600008
Download completed
http://quotes.money.163.com/trade/lsj

Download completed
http://quotes.money.163.com/trade/lsjysj_600091.html
Now downloading stock code: 600091
Download completed
http://quotes.money.163.com/trade/lsjysj_600092.html
Now downloading stock code: 600092
Download completed
http://quotes.money.163.com/trade/lsjysj_600093.html
Now downloading stock code: 600093
Download completed
http://quotes.money.163.com/trade/lsjysj_600094.html
Now downloading stock code: 600094
Download completed
http://quotes.money.163.com/trade/lsjysj_600095.html
Now downloading stock code: 600095
Download completed
http://quotes.money.163.com/trade/lsjysj_600096.html
Now downloading stock code: 600096
Download completed
http://quotes.money.163.com/trade/lsjysj_600097.html
Now downloading stock code: 600097
Download completed
http://quotes.money.163.com/trade/lsjysj_600098.html
Now downloading stock code: 600098
Download completed
http://quotes.money.163.com/trade/lsjysj_600099.html
Now downloading stock code: 600099
Download completed
http://quotes.mone

Download completed
http://quotes.money.163.com/trade/lsjysj_600177.html
Now downloading stock code: 600177
Download completed
http://quotes.money.163.com/trade/lsjysj_600178.html
Now downloading stock code: 600178
Download completed
http://quotes.money.163.com/trade/lsjysj_600179.html
Now downloading stock code: 600179
Download completed
http://quotes.money.163.com/trade/lsjysj_600180.html
Now downloading stock code: 600180
Download completed
http://quotes.money.163.com/trade/lsjysj_600181.html
Now downloading stock code: 600181
Download completed
http://quotes.money.163.com/trade/lsjysj_600182.html
Now downloading stock code: 600182
Download completed
http://quotes.money.163.com/trade/lsjysj_600183.html
Now downloading stock code: 600183
Download completed
http://quotes.money.163.com/trade/lsjysj_600184.html
Now downloading stock code: 600184
Download completed
http://quotes.money.163.com/trade/lsjysj_600185.html
Now downloading stock code: 600185
Download completed
http://quotes.mone

Download completed
http://quotes.money.163.com/trade/lsjysj_600261.html
Now downloading stock code: 600261
Download completed
http://quotes.money.163.com/trade/lsjysj_600262.html
Now downloading stock code: 600262
Download completed
http://quotes.money.163.com/trade/lsjysj_600263.html
Now downloading stock code: 600263
Download completed
http://quotes.money.163.com/trade/lsjysj_600265.html
Now downloading stock code: 600265
Download completed
http://quotes.money.163.com/trade/lsjysj_600266.html
Now downloading stock code: 600266
Download completed
http://quotes.money.163.com/trade/lsjysj_600267.html
Now downloading stock code: 600267
Download completed
http://quotes.money.163.com/trade/lsjysj_600268.html
Now downloading stock code: 600268
Download completed
http://quotes.money.163.com/trade/lsjysj_600269.html
Now downloading stock code: 600269
Download completed
http://quotes.money.163.com/trade/lsjysj_600270.html
Now downloading stock code: 600270
Download completed
http://quotes.mone

Download completed
http://quotes.money.163.com/trade/lsjysj_600349.html
Now downloading stock code: 600349
Download completed
http://quotes.money.163.com/trade/lsjysj_600350.html
Now downloading stock code: 600350
Download completed
http://quotes.money.163.com/trade/lsjysj_600351.html
Now downloading stock code: 600351
Download completed
http://quotes.money.163.com/trade/lsjysj_600352.html
Now downloading stock code: 600352
Download completed
http://quotes.money.163.com/trade/lsjysj_600353.html
Now downloading stock code: 600353
Download completed
http://quotes.money.163.com/trade/lsjysj_600354.html
Now downloading stock code: 600354
Download completed
http://quotes.money.163.com/trade/lsjysj_600355.html
Now downloading stock code: 600355
Download completed
http://quotes.money.163.com/trade/lsjysj_600356.html
Now downloading stock code: 600356
Download completed
http://quotes.money.163.com/trade/lsjysj_600357.html
Now downloading stock code: 600357
Download completed
http://quotes.mone

...

Download completed
http://quotes.money.163.com/trade/lsjysj_603059.html
Now downloading stock code: 603059
Download completed
http://quotes.money.163.com/trade/lsjysj_603060.html
Now downloading stock code: 603060
Download completed
http://quotes.money.163.com/trade/lsjysj_603063.html
Now downloading stock code: 603063
Download completed
http://quotes.money.163.com/trade/lsjysj_603066.html
Now downloading stock code: 603066
Download completed
http://quotes.money.163.com/trade/lsjysj_603067.html
Now downloading stock code: 603067
Download completed
http://quotes.money.163.com/trade/lsjysj_603069.html
Now downloading stock code: 603069
Download completed
http://quotes.money.163.com/trade/lsjysj_603076.html
Now downloading stock code: 603076
Download completed
http://quotes.money.163.com/trade/lsjysj_603077.html
Now downloading stock code: 603077
Download completed
http://quotes.money.163.com/trade/lsjysj_603078.html
Now downloading stock code: 603078
Download completed
http://quotes.mone

Download completed
http://quotes.money.163.com/trade/lsjysj_603233.html
Now downloading stock code: 603233
Download completed
http://quotes.money.163.com/trade/lsjysj_603238.html
Now downloading stock code: 603238
Download completed
http://quotes.money.163.com/trade/lsjysj_603239.html
Now downloading stock code: 603239
Download completed
http://quotes.money.163.com/trade/lsjysj_603258.html
Now downloading stock code: 603258
Download completed
http://quotes.money.163.com/trade/lsjysj_603260.html
Now downloading stock code: 603260
Download completed
http://quotes.money.163.com/trade/lsjysj_603266.html
Now downloading stock code: 603266
Download completed
http://quotes.money.163.com/trade/lsjysj_603268.html
Now downloading stock code: 603268
Download completed
http://quotes.money.163.com/trade/lsjysj_603269.html
Now downloading stock code: 603269
Download completed
http://quotes.money.163.com/trade/lsjysj_603277.html
Now downloading stock code: 603277
Download completed
http://quotes.mone

Download completed
http://quotes.money.163.com/trade/lsjysj_603477.html
Now downloading stock code: 603477
Download completed
http://quotes.money.163.com/trade/lsjysj_603488.html
Now downloading stock code: 603488
Download completed
http://quotes.money.163.com/trade/lsjysj_603496.html
Now downloading stock code: 603496
Download completed
http://quotes.money.163.com/trade/lsjysj_603499.html
Now downloading stock code: 603499
Download completed
http://quotes.money.163.com/trade/lsjysj_603500.html
Now downloading stock code: 603500
Download completed
http://quotes.money.163.com/trade/lsjysj_603501.html
Now downloading stock code: 603501
Download completed
http://quotes.money.163.com/trade/lsjysj_603505.html
Now downloading stock code: 603505
Download completed
http://quotes.money.163.com/trade/lsjysj_603506.html
Now downloading stock code: 603506
Download completed
http://quotes.money.163.com/trade/lsjysj_603507.html
Now downloading stock code: 603507
Download completed
http://quotes.mone

Download completed
http://quotes.money.163.com/trade/lsjysj_603668.html
Now downloading stock code: 603668
Download completed
http://quotes.money.163.com/trade/lsjysj_603669.html
Now downloading stock code: 603669
Download completed
http://quotes.money.163.com/trade/lsjysj_603676.html
Now downloading stock code: 603676
Download completed
http://quotes.money.163.com/trade/lsjysj_603677.html
Now downloading stock code: 603677
Download completed
http://quotes.money.163.com/trade/lsjysj_603678.html
Now downloading stock code: 603678
Download completed
http://quotes.money.163.com/trade/lsjysj_603679.html
Now downloading stock code: 603679
Download completed
http://quotes.money.163.com/trade/lsjysj_603680.html
Now downloading stock code: 603680
Download completed
http://quotes.money.163.com/trade/lsjysj_603683.html
Now downloading stock code: 603683
Download completed
http://quotes.money.163.com/trade/lsjysj_603685.html
Now downloading stock code: 603685
Download completed
http://quotes.mone

Download completed
http://quotes.money.163.com/trade/lsjysj_603859.html
Now downloading stock code: 603859
Download completed
http://quotes.money.163.com/trade/lsjysj_603860.html
Now downloading stock code: 603860
Download completed
http://quotes.money.163.com/trade/lsjysj_603861.html
Now downloading stock code: 603861
Download completed
http://quotes.money.163.com/trade/lsjysj_603866.html
Now downloading stock code: 603866
Download completed
http://quotes.money.163.com/trade/lsjysj_603868.html
Now downloading stock code: 603868
Download completed
http://quotes.money.163.com/trade/lsjysj_603869.html
Now downloading stock code: 603869
Download completed
http://quotes.money.163.com/trade/lsjysj_603871.html
Now downloading stock code: 603871
Download completed
http://quotes.money.163.com/trade/lsjysj_603877.html
Now downloading stock code: 603877
Download completed
http://quotes.money.163.com/trade/lsjysj_603878.html
Now downloading stock code: 603878
Download completed
http://quotes.mone